<a href="https://colab.research.google.com/github/tamirdh/Deep-learning-image-caption/blob/master/image_captioning_deep_learning_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloads

In [5]:
%cd /home/yandex/DLW2021/davidhay/Deep-learning-image-caption/
!pwd
!ls -l 

/home/yandex/DLW2021/davidhay/Deep-learning-image-caption
/home/yandex/DLW2021/davidhay/Deep-learning-image-caption
total 6596
drwxr-xr-x 4 davidhay math1    4096 Aug 22 22:00 coco
-rw-r--r-- 1 davidhay math1 6654519 Aug 22 00:54 image_captioning_deep_learning_workshop.ipynb
-rw-r--r-- 1 davidhay math1      74 Aug 20 22:36 README.md
-rw-r--r-- 1 davidhay math1      15 Aug 21 19:14 test.py
-rw-r--r-- 1 davidhay math1     197 Aug 22 01:02 test.slurm
-rw-r--r-- 1 davidhay math1   45488 Aug 21 18:55 vocab.pkl


In [ ]:
!rm -rf coco
!mkdir coco
%cd coco
!mkdir images
%cd images
!mkdir train2017

In [ ]:
!wget -c http://images.cocodataset.org/zips/train2017.zip
!wget -c http://images.cocodataset.org/zips/val2017.zip

!unzip train2017.zip
!unzip val2017.zip

!rm train2017.zip
!rm val2017.zip

%cd ../
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip

!unzip annotations_trainval2017.zip

!rm annotations_trainval2017.zip

# Data Loading

## Imports and Vocabulary 

In [7]:
import os
from collections import Counter
import spacy
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from pycocotools.coco import COCO
from PIL import Image
import pickle

spacy_eng = spacy.load("en_core_web_sm")

class Vocabulary:
    def __init__(self,freq_threshold):
        #setting the pre-reserved tokens int to string tokens
        # PAD- padding symbol
        # SOS- Start of Sentence
        # EOS- end of sentence
        # UNK- unknown word (unknown\ below threshold)
        self.itos = {0:"<PAD>",1:"<SOS>",2:"<EOS>",3:"<UNK>"}
        #string to int tokens
        #its reverse dict self.itos
        self.stoi = {v:k for k,v in self.itos.items()}
        self.freq_threshold = freq_threshold
        
    def __len__(self):
      return len(self.itos)
    
    @staticmethod
    def tokenize(text):
        return [token.text.lower() for token in spacy_eng.tokenizer(text)]
    
    def build_vocab(self, sentence_list):
        frequencies = Counter()
        idx = 4
        for index,sentence in enumerate(sentence_list):

            for word in self.tokenize(sentence):
                frequencies[word] += 1
                
                #add the word to the vocab if it reaches minum frequecy threshold
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    if idx > 0 and idx % 1000==0:
                        print(f"Added {idx} words to vocab")
                    idx += 1
            if index>0 and index%1000==0:
                print(f"Iterated {index} sentences")
             

        print(f"Done, added {idx-1} words to vocabulary")
    
    def numericalize(self,text):
        """ For each word in the text corresponding index token for that word form the vocab built as list """
        tokenized_text = self.tokenize(text)
        result = [ self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text ]
        return result

## Dataset custom class

In [8]:
import pickle
class COCODataset(Dataset):
    """
    COCODataset
    """
    def __init__(self,root_dir,annotation_file,transform=None,freq_threshold=5,
                 load_vocab=False, vocab_loc = "vocab.pkl"):
      """
      can use load_vocab to use a previously created vocabulary (time saving feature)
      freq_threshold: words with a count below this number will be marked as <UNK>
      """
      self.root_dir = root_dir
      self.coco = COCO(annotation_file)
      self.transform = transform
      self.cap_max_size = 0
      #Get image and caption colum from the dataframe
      self.imgs = []
      self.captions = []
      for idx,ann in enumerate(self.coco.anns.values()):
        self.imgs.append(self.coco.loadImgs((ann['image_id']))[0]["file_name"])
        self.captions.append(ann['caption'])
        if (idx) % 1000 == 0 and idx>0:
          print(f"Processed {idx} images and captions")
      print("Finished processing images and captions")
      print(f"Got:{len(set(self.imgs))} pictures with {len(self.captions)} captions!")
      
      #Initialize vocabulary and build vocab
      if load_vocab:
        with open(vocab_loc, "rb") as source:
          self.vocab = pickle.load(source)
        print(f"Loaded vocabulary from {vocab_loc}")
      
      else:
        print("Build vocabulary")
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocab(self.captions)
        print("Finished building vocabulary")
        with open(vocab_loc, "wb") as dest:
          pickle.dump(self.vocab, dest)
      
      print(f"Using {len(self.vocab)} words")
    
    def __len__(self):
      return len(self.imgs)
    
    def __getitem__(self,idx):
      caption = self.captions[idx]
      img_name = self.imgs[idx]
      img_location = os.path.join(self.root_dir,img_name)
      img = Image.open(img_location).convert("RGB")
      
      #apply the transfromation to the image
      if self.transform:
          img = self.transform(img)
      
      #numericalize the caption text
      caption_vec = [self.vocab.stoi["<SOS>"]]
      caption_vec.extend(self.vocab.numericalize(caption))
      caption_vec.append(self.vocab.stoi["<EOS>"])
      
      return img, torch.tensor(caption_vec,dtype=torch.long)

## Dataloader creation

In [9]:
# define a transformation to add some noise and variance to our images
transformation = transforms.Compose([transforms.Resize((512,512), Image.NEAREST),
                                     transforms.ToTensor(),
                                     transforms.RandomVerticalFlip(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                      ])

In [10]:
class CapsCollate:
    """
    Collate to apply the padding to the captions with dataloader
    """
    def __init__(self,pad_idx,batch_first=False, vec_len=-1):
        self.pad_idx = pad_idx
        self.batch_first = batch_first
        self.vec_len = vec_len + 2 # adding the <SOS> and <EOS>
        assert self.vec_len > 0, "Vector length must be positive integer"
    
    def __call__(self,batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs,dim=0)
        targets_list = list()
        for item in batch:
            # item = (img:Image, caption:tensor)
            addition = self.vec_len-len(item[1])
            padded_target = torch.cat((item[1], torch.empty(addition,dtype=torch.long).fill_(pad_idx)),dim=0)
            targets_list.append(padded_target)
            #print(f"GOT:{item[1]}, {item[1].type()}\nAdding:{addition}\nPADDED:{padded_target}\n{padded_target.type()}")
        targets = torch.stack(targets_list,0)
        #print(f"Targets shape:{targets.shape}")
        return imgs,targets

In [11]:

dataset =  COCODataset(
    root_dir = "/home/yandex/DLW2021/davidhay/Deep-learning-image-caption/coco/images/train2017",
    annotation_file= "/home/yandex/DLW2021/davidhay/Deep-learning-image-caption/coco/annotations/captions_train2017.json",
    transform=transformation,
    freq_threshold=5,
    load_vocab=False
)

loading annotations into memory...
Done (t=1.55s)
creating index...
index created!
Processed 1000 images and captions
Processed 2000 images and captions
Processed 3000 images and captions
Processed 4000 images and captions
Processed 5000 images and captions
Processed 6000 images and captions
Processed 7000 images and captions
Processed 8000 images and captions
Processed 9000 images and captions
Processed 10000 images and captions
Processed 11000 images and captions
Processed 12000 images and captions
Processed 13000 images and captions
Processed 14000 images and captions
Processed 15000 images and captions
Processed 16000 images and captions
Processed 17000 images and captions
Processed 18000 images and captions
Processed 19000 images and captions
Processed 20000 images and captions
Processed 21000 images and captions
Processed 22000 images and captions
Processed 23000 images and captions
Processed 24000 images and captions
Processed 25000 images and captions
Processed 26000 images and

Processed 477000 images and captions
Processed 478000 images and captions
Processed 479000 images and captions
Processed 480000 images and captions
Processed 481000 images and captions
Processed 482000 images and captions
Processed 483000 images and captions
Processed 484000 images and captions
Processed 485000 images and captions
Processed 486000 images and captions
Processed 487000 images and captions
Processed 488000 images and captions
Processed 489000 images and captions
Processed 490000 images and captions
Processed 491000 images and captions
Processed 492000 images and captions
Processed 493000 images and captions
Processed 494000 images and captions
Processed 495000 images and captions
Processed 496000 images and captions
Processed 497000 images and captions
Processed 498000 images and captions
Processed 499000 images and captions
Processed 500000 images and captions
Processed 501000 images and captions
Processed 502000 images and captions
Processed 503000 images and captions
P

Iterated 150000 sentences
Iterated 151000 sentences
Iterated 152000 sentences
Iterated 153000 sentences
Iterated 154000 sentences
Iterated 155000 sentences
Iterated 156000 sentences
Iterated 157000 sentences
Iterated 158000 sentences
Iterated 159000 sentences
Iterated 160000 sentences
Iterated 161000 sentences
Iterated 162000 sentences
Iterated 163000 sentences
Iterated 164000 sentences
Iterated 165000 sentences
Iterated 166000 sentences
Iterated 167000 sentences
Iterated 168000 sentences
Iterated 169000 sentences
Iterated 170000 sentences
Iterated 171000 sentences
Added 6000 words to vocab
Iterated 172000 sentences
Iterated 173000 sentences
Iterated 174000 sentences
Iterated 175000 sentences
Iterated 176000 sentences
Iterated 177000 sentences
Iterated 178000 sentences
Iterated 179000 sentences
Iterated 180000 sentences
Iterated 181000 sentences
Iterated 182000 sentences
Iterated 183000 sentences
Iterated 184000 sentences
Iterated 185000 sentences
Iterated 186000 sentences
Iterated 187

Iterated 462000 sentences
Iterated 463000 sentences
Iterated 464000 sentences
Iterated 465000 sentences
Iterated 466000 sentences
Iterated 467000 sentences
Iterated 468000 sentences
Iterated 469000 sentences
Iterated 470000 sentences
Iterated 471000 sentences
Iterated 472000 sentences
Iterated 473000 sentences
Iterated 474000 sentences
Iterated 475000 sentences
Iterated 476000 sentences
Iterated 477000 sentences
Iterated 478000 sentences
Iterated 479000 sentences
Iterated 480000 sentences
Iterated 481000 sentences
Iterated 482000 sentences
Iterated 483000 sentences
Iterated 484000 sentences
Iterated 485000 sentences
Iterated 486000 sentences
Iterated 487000 sentences
Iterated 488000 sentences
Iterated 489000 sentences
Iterated 490000 sentences
Iterated 491000 sentences
Iterated 492000 sentences
Iterated 493000 sentences
Iterated 494000 sentences
Iterated 495000 sentences
Iterated 496000 sentences
Iterated 497000 sentences
Iterated 498000 sentences
Iterated 499000 sentences
Iterated 500

In [ ]:
print(dataset.vocab.stoi) 

In [12]:

BATCH_SIZE = 1
NUM_WORKER = 1
#token to represent the padding
pad_idx = dataset.vocab.stoi["<PAD>"]

data_loader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=False,
    collate_fn=CapsCollate(pad_idx=pad_idx,batch_first=True, vec_len=75)
)

# Models

In [13]:
import torch
import torch.nn as nn
import torchvision.models as models

device = "cuda" if torch.cuda.is_available() else "cpu"

class EncoderCNN(nn.Module):
  def __init__(self, embed_size, train_CNN=False):
      super(EncoderCNN, self).__init__()
      self.train_CNN = train_CNN
      self.inception = models.inception_v3(pretrained=True, aux_logits=False)
      self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
      self.relu = nn.ReLU()
      self.dropout = nn.Dropout(0.5)

  def forward(self, images):
      
      features = self.inception(images)
      output = self.dropout(self.relu(features))
      return output


class DecoderRNN(nn.Module):
  """
  Input is a CNN network, output will be a caption.
  TODO: Check how to implement a transformer for better results
  """
  def __init__(self, embed_size, hidden_size, vocab_size):
      super(DecoderRNN, self).__init__()
      self.hidden_size = hidden_size
      self.vocab_size = vocab_size
      self.embed = nn.Embedding(vocab_size, embed_size)
      self.lstm_cell = nn.LSTMCell(embed_size, hidden_size)
      self.fc_out = nn.Linear(hidden_size, vocab_size)
      self.dropout = nn.Dropout(0.5)

  def forward(self, features, captions, show=False):
        # batch size
        batch_size = features.size(0)
        
        
        # init the hidden and cell states to zeros
        hidden_state = torch.zeros((batch_size, self.hidden_size)).to(device)
        cell_state = torch.zeros((batch_size, self.hidden_size)).to(device)
        hidden_state, cell_state = self.lstm_cell(features, (hidden_state, cell_state))
        # define the output tensor placeholder
        outputs = torch.empty((batch_size, captions.size(1), self.vocab_size)).to(device)

        # embed the captions
        captions_embed = self.embed(captions)
        # tensor of shape (B, LEN, EMBED SIZE)
        # LEN- vectors length (longest caption+2)
        
        # pass the caption word by word
        for t in range(captions.size(1)):

            # for the first time step the input is the feature vector
            # if t == 0:
            #     hidden_state, cell_state = self.lstm_cell(features, (hidden_state, cell_state))
                
            # # for the 2nd+ time step, using teacher forcer
            # else:
            hidden_state, cell_state = self.lstm_cell(captions_embed[:, t, :], (hidden_state, cell_state))
            # output of the attention mechanism
            out = self.fc_out(self.dropout(hidden_state))
            # build the output tensor
            outputs[:, t, :] = out
        if show:
            #print(f"Captions:{captions}")
            #print(f"outputs shape:{outputs.shape}")
            pass
        return outputs

  

class CNNtoRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, train_CNN=False):
      super(CNNtoRNN, self).__init__()
      self.encoderCNN = EncoderCNN(embed_size, train_CNN).to(device)
      self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size).to(device)
      #self.decoderRNN = DecoderRNNConcat(embed_size, hidden_size, vocab_size).to(device)

  def forward(self, images, captions, show=False):
      features = self.encoderCNN(images)
      outputs = self.decoderRNN(features, captions, show)
      return outputs
  def caption_images(self, image, vocab, max_len=50):
      # Inference part
      # Given the image features generate the captions
      # input shape: (3,x,y) where, x,y: image size
      # ouput: captions list
    batch_size = image.size(0)
    assert batch_size == 1, "Caption 1 image at a time"
    image_pred = self.encoderCNN(image)

    # init the hidden and cell states to zeros
    hidden_state = torch.zeros((1, self.decoderRNN.hidden_size)).to(device)
    cell_state = torch.zeros((1, self.decoderRNN.hidden_size)).to(device)
    
    #starting input is 
    captions = list()
    outputs = torch.empty((batch_size, max_len, self.decoderRNN.vocab_size)).to(device)
    hidden_state, cell_state = self.decoderRNN.lstm_cell(image_pred, (hidden_state, cell_state))
    out = self.decoderRNN.fc_out(self.decoderRNN.dropout(hidden_state))
    word_embed = self.decoderRNN.embed(torch.tensor(vocab.stoi["<SOS>"]).to(device)).unsqueeze(0)
    for t in range(max_len):
        # for the first time step the input is the feature vector
        # if t == 0:
        #     hidden_state, cell_state = self.decoderRNN.lstm_cell(image_pred, (hidden_state, cell_state))
        # for the 2nd+ time step, use previously generated caption
        # else:
        hidden_state, cell_state = self.decoderRNN.lstm_cell(word_embed, (hidden_state, cell_state))
        
        # output of the attention mechanism
        out = self.decoderRNN.fc_out(hidden_state)
        outputs[:, t, :] = out
        #print(f"out shape:{out.shape}")
        captions.append(torch.argmax(out,dim=1))
        #print(f"\n predicted outputs:{captions}")
        word_embed = self.decoderRNN.embed(torch.argmax(out[0])).unsqueeze(0)
        last_word_idx = captions[-1].item()
        if vocab.itos[last_word_idx] == vocab.stoi["<EOS>"]:
            print("BREAKING!!!!!!")
            break
            
            
            # build the output tensor
        #print(captions)
        #covert the vocab idx to words and return sentence
    print(f"outputs shape:{outputs.shape}")
    print(f"Outputs argmax dim2:{torch.argmax(outputs,dim=2)}")
    return [vocab.itos[idx.item()] for idx in captions if idx.item() != vocab.stoi["<PAD>"]]
  

# Training

## training function

In [14]:
import torch.optim as optim
from tqdm import tqdm


def train(max_epochs, model):
  # Hyperparameters
  learning_rate = 3e-4
  device = "cuda" if torch.cuda.is_available() else "cpu"

  
  # init model
  model = model.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  model.train()
     

  # start epochs
  for epoch in range(max_epochs):
    for idx, (img, captions) in tqdm(
            enumerate(data_loader), total=len(data_loader), leave=False
        ):
      img = img.to(device)
      captions = captions.to(device).long()
      output = model(img, captions).to(device)
      loss = criterion(output.reshape(-1, output.shape[2]), captions.reshape(-1))
      optimizer.zero_grad()
      loss.backward(loss)
      optimizer.step()

      if idx>0 and idx%100==0:
        dataiter = iter(data_loader)
        img_show,cap = next(dataiter)
        print(f"Loss {loss.item():.5f}\n")
        demo_cap = model.caption_images(img_show[0:1].to(device), vocab=dataset.vocab, max_len=30)
        demo_cap = ' '.join(demo_cap)
        print("Predicted")
        show_image(img_show[0],title=demo_cap)
        print("Original")
        cap = cap[0]
        print(cap.long())
        demo_cap = ' '.join([dataset.vocab.itos[idx2.item()] for idx2 in cap if idx2.item() != dataset.vocab.stoi["<PAD>"]])
        show_image(img_show[0],title=demo_cap, transform=False)
        

  return model

## image function

In [15]:
import matplotlib.pyplot as plt
def show_image(img, title=None, transform=True):
    """Imshow for Tensor."""
    
    #unnormalize 
    if transform:
      img[0] = img[0] * 0.229
      img[1] = img[1] * 0.224 
      img[2] = img[2] * 0.225 
      img[0] += 0.485 
      img[1] += 0.456 
      img[2] += 0.406
      
    img = img.numpy().transpose((1, 2, 0))
    
    
    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

## Overfit sanity check

In [16]:
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)
def overfit(model, T=250):
    learning_rate = 3e-4
    device = "cuda" if torch.cuda.is_available() else "cpu"

    
    # init model
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()

    
    dataiter = iter(data_loader)
    img,caption = next(dataiter)
    for i in tqdm(range(T)):
        # train on the same image and caption to achieve overfitting
        img = img.to(device)
        caption = caption.to(device).long()
        output = model(img, caption, show=False).to(device)
        loss = criterion(output.reshape(-1, output.shape[2]), caption.reshape(-1))
        optimizer.zero_grad()
        loss.backward(loss)
        optimizer.step()
    output = model(img, caption, show=False).to(device)
    show_img = img.to("cpu")
    print(f"\n\nLoss {loss.item():.5f}\n")
    #print(f"\nForward\n")
    out_cap = torch.argmax(output[0],dim=1)
    #print(f"Forwad num vals:{out_cap}")
    demo_cap = ' '.join([dataset.vocab.itos[idx2.item()] for idx2 in out_cap if idx2.item() != dataset.vocab.stoi["<PAD>"]])
    show_image(show_img[0],title=demo_cap)
    print("Predicted")
    with torch.no_grad():
        model.eval()
        demo_cap = model.caption_images(show_img[0:1].to(device), vocab=dataset.vocab, max_len=15)
        demo_cap = ' '.join(demo_cap)
        model.train()
        
        show_image(show_img[0],title=demo_cap, transform=False)
    print("Original")
    cap = caption[0]
    #print(cap.long())
    demo_cap = ' '.join([dataset.vocab.itos[idx2.item()] for idx2 in cap if idx2.item() != dataset.vocab.stoi["<PAD>"]])
    show_image(show_img[0],title=demo_cap, transform=False)

        


## Process

In [ ]:
embed_size = 300
hidden_size = 5000
vocab_size = len(dataset.vocab)
model = CNNtoRNN(embed_size, hidden_size, vocab_size, train_CNN=False)
trained_model = train(3, model)

# Misc

In [17]:
embed_size = 400
hidden_size = 3000
vocab_size = len(dataset.vocab)
model = CNNtoRNN(embed_size, hidden_size, vocab_size, train_CNN=False)
overfit(model,250)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /home_dir/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
for imgs,caps in data_loader:
    print(f"\ncap shape:{caps.shape}")
    print(f"\nFirst caption:{caps[0]}")
    print(f"\nFirst embed:{caps[0][0]}")
    break

In [24]:
!conda install -c conda-forge -y ipywidgets 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - ipywidgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    argon2-cffi-20.1.0         |   py38h25fe258_2          47 KB  conda-forge
    async_generator-1.10       |             py_0          18 KB  conda-forge
    bleach-4.0.0               |     pyhd8ed1ab_0         111 KB  conda-forge
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py38h578d9bd_0         141 KB  conda-forge
    conda-4.10.3               |   py38h578d9bd_0         3.1 MB  conda-forge
    defusedxml-0.7.1           |     pyhd8ed1ab_0          23 KB  conda-forge
 

In [13]:
!source /home/yandex/DLW2021/davidhay/anaconda3/bin/activate
!conda info

_CONDA_ROOT=/home/yandex/DLW2021/davidhay/anaconda3: Command not found.
/opt/conda/bin/.: Permission denied.
return: Command not found.
Illegal variable name.

     active environment : None
            shell level : 0
       user config file : /home_dir/.condarc
 populated config files : 
          conda version : 4.9.2
    conda-build version : 3.20.5
         python version : 3.8.5.final.0
       virtual packages : __cuda=11.2=0
                          __glibc=2.31=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /opt/conda  (read only)
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /opt/conda/pkgs
                          /home_dir/.conda/pkgs
       envs directories : 

In [7]:
!conda activate /home/yandex/DLW2021/davidhay/anaconda3/envs/new-env
!conda info --envs




# conda environments:
#
                         /home/yandex/DLW2021/davidhay/anaconda3
                         /home/yandex/DLW2021/davidhay/anaconda3/envs/new-env
base                  *  /opt/conda



In [1]:

!conda info --envs

# conda environments:
#
base                  *  /home/yandex/DLW2021/davidhay/anaconda3
new-env                  /home/yandex/DLW2021/davidhay/anaconda3/envs/new-env



In [9]:
!conda install -y ipykernel
!python kernel install --user --name=new-env

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - ipykernel


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.7.5   |       h06a4308_1         113 KB
    certifi-2021.5.30          |   py38h06a4308_0         138 KB
    conda-4.10.3               |   py38h06a4308_0         2.9 MB
    ipykernel-5.3.4            |   py38h5ca1d4c_0         183 KB
    jupyter_client-6.1.12      |     pyhd3eb1b0_0          88 KB
    jupyter_core-4.7.1         |   py38h06a4308_0          68 KB
    libsodium-1.0.18           |       h7b6447c_0         244 KB
    python-dateutil-2.8.2      |     pyhd3eb1b0_0         233 KB
    pyzmq-22.2.1               |   py38h295c915_1         465 KB
    tornado-6.1                |   py38h27cfd23_0         588 KB
    ------------------------------------------------------------
               